In [1]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [3]:
import sys
sys.path.append("../")
import os
import datetime
import pandas as pd
import pandas_market_calendars as mcal
from polygon import RESTClient
import time
import pytz
eastern = 'US/Eastern'

In [12]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_split_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_splits(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting split data for "+ str(stock))
            return []

def get_dividend_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_dividends(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting dividend data for "+ str(stock))
            return []
        
def extract_qualified_symbols(df, plvl, advlvl):
    qualified = df[(df.p_lvl == plvl) & (df.adv_lvl == advlvl)]
    return qualified

In [13]:
main_df = pd.read_csv('stocks-profile.csv')


In [15]:
q_sym_list = []
for i in range(4):
    q_sym = extract_qualified_symbols(main_df, i, 2)
    q_sym_list = q_sym_list + list(q_sym['symbol'])


In [16]:
splits_ = []
dividends_ = []

In [30]:
for i in tqdm(range(len(q_sym_list))):
    
    ticker = q_sym_list[i]
    
    sp = get_split_data_for_stock(ticker)
    div = get_dividend_data_for_stock(ticker)
    
    splits_.append({
        'symbol':ticker,
        'splits':sp
    })
    
    dividends_.append({
        'symbol':ticker,
        'dividends':div
    })
    
    time.sleep(0.1)



<ipython-input-30-ba7edb74297d>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(q_sym_list))):


In [31]:
splits_df = pd.DataFrame(splits_)
divs_df = pd.DataFrame(dividends_)

In [32]:
splits_df.to_csv('splits_data.csv')
divs_df.to_csv('dividends_data.csv')

# BECAUSE THE STOCKS HAD VARYING NUMBERS WHEN IT CAME TO SPLITS AND DIVIDENS, I STORED THE VALUES AS LISTS OF DICTIONARIES

In [28]:
type(splits_df.iloc[2]['splits'])

list

In [33]:
splits_df.head()

,symbol,splits
0,SNDL,[]
1,GNUS,"[{'ticker': 'GNUS', 'exDate': '2016-11-09', 'p..."
2,NOK,"[{'ticker': 'NOK', 'exDate': '2000-04-11', 'pa..."
3,TOPS,"[{'ticker': 'TOPS', 'exDate': '2020-08-10', 'p..."
4,IBIO,"[{'ticker': 'IBIO', 'exDate': '2018-06-11', 'p..."


In [34]:
splits_df['splits'][2]

[{'ticker': 'NOK',
  'exDate': '2000-04-11',
  'paymentDate': '2000-04-11',
  'ratio': 0.25},
 {'ticker': 'NOK',
  'exDate': '1999-04-12',
  'paymentDate': '1999-04-12',
  'ratio': 0.5},
 {'ticker': 'NOK',
  'exDate': '1998-04-16',
  'paymentDate': '1998-04-16',
  'ratio': 0.5}]